<a href="https://colab.research.google.com/github/fjme95/python-para-la-ciencia-de-datos/blob/main/Semana%207/Introducci%C3%B3n_a_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El contenido de este notebook es una traducción casi literal de [Intro to Keras for Engineers](https://keras.io/getting_started/intro_to_keras_for_engineers/). 

# Dependencias

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Introducción


En este notebook se revisaran los pasos para

- Preparar los datos antes de entrenar el modelo. Hacer preprocesamiento de lso datos.
- Construir un modelo.
- Entrenar el modelo con el método ```fit()```.
- Evaluar el modelo con datos no vistos durante el entrenamiento.



# Carga y preprocesamiento de los datos

Las redes neuronales no procesan datos crudos como archivos de texto, imágenes codificadas, o archivos csv. Procesan representaciones **vectorizadas** y **estandarizadas**.

- Los archivos de texto tienen que ser cargados en tensores de texto, luego dividos en palabras. Finalmente, las palabras tienen que ser indexadas y convertidas a tensores de números.
- Las imágenes tienen que ser leías y decodificadas en tensores de números, convertidas a tensores de punto flotante y normalizadas a valores pequeños (usualmente a valores entre 0 y 1).
- Datos en csv tiene que ser parseados, con variables númericas convertidas a tensores de punto flotante y variables categóricas indexadas y convertidas a tensores de enteros. Luego, cada característica es usualmente normalizada para tener media cero y varianza uno.
- Etc.

Comencemos con la carga de los datos.

## Carga de los datos

Los modelos de Keras aceptan tres tipos de entrada.

- **Arreglos de numpy**: Son una buena opción si los datos caben en memoria.
- **Objectos ```Tensorflow.Dataset```**: Es una opción con buen rendimiento cuando el dataset no cabe en memoría que son transmitidos desde el disco o desde un sistema de archivos distribuido.
- **Generadores de python**: que dan lotes de datos.

Si se tiene un dataset lo suficiente grande para que no quepa en memoria y se está entrenando el modelo en la GPU, considere usar objetos de tipo ```Dataset```, pues se harán cargo de detalles críticos de rendimiento, como, 
- Preprocesar los datos asíncronamente en CPU mientras la GPU esta ocupada, y almacenándolos en una cola.
- Precargando datos en GPU para que estén disponibles inmediatamente en cuanto la GPU haya terminado de procesar el lote previo.

Keras tiene varias utilidades para convertir datos crudos en el disco a un ```Dataset```:
- ```tf.keras.preprocessing.image_dataset_from_directory```convierte archivos de imágenes acomodadoes en directorios específicos por clase a un dataset de tensores de imágenes.
- ```tf.keras.preprocessing.text_dataset_from_directory``` hace lo mismo para archivos de texto


## Ejemplo: Obtener un dataset etiquetado desde imágenes en el disco.

In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: tiny-imagenet-200/val/images/val_3979.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6982.JPEG  
  inflating: tiny-imagenet-200/val/images/val_8496.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7332.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9241.JPEG  
  inflating: tiny-imagenet-200/val/images/val_4196.JPEG  
  inflating: tiny-imagenet-200/val/images/val_5980.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6697.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9969.JPEG  
  inflating

En este ejemplo vamos a tener imágenes ordenadas en distintos directorios como

```
datos/
    train/
        label_a/
            images/
                a_image_1.jpeg
                a_image_2.jpeg
                ...
        label_b/
            images/
                b_image_1.jpeg
                b_image_2.jpeg
                ...
        ...
    test/
        images/
            test_image_0.jpeg
            test_image_1.jpeg
            ...
```

Lo podemos cargar de la siguiente manera:

In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
  '/content/tiny-imagenet-200/train', batch_size=512, image_size=(64, 64))

# Para demostración, itera sobre los lotes producidos por el dataset
for i, (data, labels) in enumerate(dataset):
   print(data.shape)  # (64, 200, 200, 3)
   print(data.dtype)  # float32
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32
   if i == 10:
       break

Found 100000 files belonging to 200 classes.
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>
(512, 64, 64, 3)
<dtype: 'float32'>
(512,)
<dtype: 'int32'>


Las etiquetas, ```labels```, de una muestra es el rango del folder en el que se encuentra la imagen en orden alfanumérico.

Para un dataset de texto se haría algo similar.

## Preprocesamiento usando capas de keras.

Una vez que los datos se encuentran en un arreglo de numpy de cadenas/enteros/flotantes o en un objeto de tipo ```Dataset``` que de lotes de tensores de cadenas/enteros/flotantes hay que preprocesar los datos. Esto puede significar:

- Tokenizar datos de texto, seguido de indexado de tokens.
- Normalización de las características.
- Reescalamiento de los datos a valores pequeños (generalmente, los valores de entrada para una red neuronal deben ser ceranos a cero, tipicamente se espera que los datos tengan media cero y varianza uno o que los valores estén entre 0 y 1).


## Usando las capas de preprocesamiento de Keras.

En keras se puede hacer el preprocesamiento de los datos usando las capas de preprocesamiento. Estas incluyen, 
- Vectorizacion de texto usando la capa ```TextVectorization```.
- Normalización de características usando la capa ```Normalization```.
- Reescalamiento de imágenes, cortado, o aumentación de los datos.

La ventaja de usar las capas de preprocesamiento de keras es que **pueden ser incluidas directo en el modelo**, ya sea durante o depués de del entrenamiento, haciendo los modelos pórtatiles.

Algunas capas de preprocesamiento mantienen un estado:
- ```TextVectorization``` mantiene el mapeo del índice a palabras o de tokens a índices.
- ```Normalization``` mantiene la media y varianza de las características.

El estado de la capa de procesamiento se obtiene llamando ```layer.adapt(data)```

## Ejemplo: Convirtiendo cadenas a secuencias de indices de palabras.

In [ ]:
from tensorflow.keras.layers import TextVectorization

# Datos de entrenamiento de muestra de dtype string
training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

# Crea una instancia de la capa TextVectorization. Puede ser configurada para 
# regresar los índices de los tokens, o una representación densa de los tokens
# (e.g tf-idf). Los algoritmos de estandarización y división del texto son 
# completamente configurables.
vectorizer = TextVectorization(output_mode="int")

# Llamar `adapt` en el arreglo o el dataset hace que la capa genere un índice
# del vocabulario para los datos, que después puede ser reutilizado cuando
# vea nuevos datos.
vectorizer.adapt(training_data)

# Después de llamar adapt, la capa es capaz de codificar cualquier n-grama que
# haya visto previamente cuando de adapto a los datos. n-gramas desconocidos son
# codificados con un "out-of-vocabulary" token.
integer_data = vectorizer(training_data)
print(integer_data)


tf.Tensor(
[[4 5 2 9 3]
 [7 6 2 8 3]], shape=(2, 5), dtype=int64)


In [ ]:
vectorizer.get_vocabulary()

['', '[UNK]', 'the', 'sample', 'this', 'is', 'heres', 'and', '2nd', '1st']

## Ejemplo: Convirtiendo cadenas a secuencias de bigramas con One-Hot-Encoding.

In [ ]:
from tensorflow.keras.layers import TextVectorization

# Datos de entrenamiento de muestra de dtype string
training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."], ["Finally, here's the 3rd sample"]])

# Crea una instancia de la capa TextVectorization. Puede ser configurada para 
# regresar los índices de los tokens, o una representación densa de los tokens
# (e.g tf-idf). Los algoritmos de estandarización y división del texto son 
# completamente configurables.
vectorizer = TextVectorization(output_mode="binary", ngrams=2)

# Llamar `adapt` en el arreglo o el dataset hace que la capa genere un índice
# del vocabulario para los datos, que después puede ser reutilizado cuando
# vea nuevos datos.
vectorizer.adapt(training_data)

# Después de llamar adapt, la capa es capaz de codificar cualquier n-grama que
# haya visto previamente cuando de adapto a los datos. n-gramas desconocidos son
# codificados con un "out-of-vocabulary" token.
tfidf_data = vectorizer(training_data)
print(tfidf_data)


tf.Tensor(
[[0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.]
 [0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0.]], shape=(3, 22), dtype=float32)


In [ ]:
vectorizer.get_vocabulary()

['[UNK]',
 'the',
 'sample',
 'heres the',
 'heres',
 'this is',
 'this',
 'the 3rd',
 'the 2nd',
 'the 1st',
 'is the',
 'is',
 'finally heres',
 'finally',
 'and heres',
 'and',
 '3rd sample',
 '3rd',
 '2nd sample',
 '2nd',
 '1st sample',
 '1st']

## Ejemplo: Normalizando características

In [ ]:
from tensorflow.keras.layers import Normalization

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

normalizer = Normalization(axis=-1)
normalizer.adapt(training_data)

normalized_data = normalizer(training_data)
print("var: %.4f" % np.var(normalized_data))
print("mean: %.4f" % np.mean(normalized_data))


var: 1.0000
mean: 0.0000


## Ejemplo: Normalizando y cortando imágenes desde el centro

In [ ]:
from tensorflow.keras.layers import CenterCrop
from tensorflow.keras.layers import Rescaling

# Example image data, with values in the [0, 255] range
training_data = data

cropper = CenterCrop(height=32, width=32)
scaler = Rescaling(scale=1.0 / 255)

output_data = scaler(cropper(training_data))
print("shape:", output_data.shape)
print("min:", np.min(output_data))
print("max:", np.max(output_data))


shape: (512, 32, 32, 3)
min: 0.0
max: 1.0


In [ ]:
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
fig = make_subplots(1, 2)

fig.add_trace(px.imshow(training_data[0]).data[0], 1, 1)
fig.add_trace(px.imshow(output_data[0]).data[0], 1, 2)

fig.show()

# Construyendo modelos usando el API funcional de Keras

Una "capa" es una transformación que tiene una entrada y una salida. Por ejemplo, esta capa es una proyección lineal que mapea la entrada a un espacio de 16 dimensiones.

In [ ]:
dense = keras.layers.Dense(units=16)

Un "modelo" es un grafo acíclico dirigido de capas. Se puede ver como una "capa grande" constituida por multiples subcapas que pueden ser entrenadas cuando se exponen a los datos.

La manera más común de contruir modelos con Keras es usando la Functional API. Para contruir modelos, se empieza especificando la forma (dimensionalidad) de los datos de entrada (y opcionalmente el dtype). Si la forma de la entrada puede variar, se puede especificar como ```None```. Por ejemplo, una imagen RGB de 200x200 tendría la forma ```(200, 200, 3)```, pero una imagen de cualquier tamaño sería ```(None, None, 3)```.

In [ ]:
# Para el dataset de tiny imagenet, las imagenes son de tamaño 64x64
inputs = keras.Input(shape=(64, 64, 3))


Después de definir la entrada, se pueden "encadenar" las capas de transformación hasta la salida del modelo.

In [ ]:
from tensorflow.keras import layers

# Center-crop images to 32x32
x = CenterCrop(height=32, width=32)(inputs)
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)

# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 200
outputs = layers.Dense(num_classes, activation="softmax")(x)


Una vez que se ha definido el grafo acíclico que convierte las entradas en salidas, instancia un objeto de tipo ```Model```.

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)

El modelo se comporta como una capa grande. Se puede llamar en lotes de datos de la siguiente manera:

In [ ]:
processed_data = model(data)
print(processed_data.shape)
# np.argmax(processed_data, 1)

(512, 200)


Se puede imprimir cómo los datos son transformados en cada paso del modelo. Esto es útil para "debugear".

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 center_crop_3 (CenterCrop)  (None, 32, 32, 3)         0         
                                                                 
 rescaling_3 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 8, 8, 32)          9248      
                                                           

La Functional API también hace fácil construir modelos que tienen distintas entradas (por ejemplo, una imagen y su metadata) o múltiples salidas (por ejemplo, predecir la clase de una imagen y la verosimilitud de que un usuario haga click en ella).

# Entrenando modelos con ```fit()```

En este punto se ha visto:

- Cómo preparar los datos (e.g. como un arreglo de numpy o un objeto ```tf.data.Dataset```)
- Cómo construir un modelo que procese los datos.

El siguiente paso es entrenar el modelo con los datos. La clase ```modelo``` tiene contruido un ciclo de entrenamiento, el método ```fit()```. Este acepta objetos de tipo ```Dataset```, arreglos de numpy o generadores de python que suministren lotes de datos.

Antes de llamar ```fit()```, se tiene que especificar un optimizador y una función de pérdida. Este es el paso ```compile()```:

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy())

La pérdida y el optimizador pueden ser especificados con sus identificadores de cadena (en este caso son usados los valores por default del constructor):

```python
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
```

Una vez que el modelo esta compilado, se puede comenzar el "ajuste" del modelo a los datos. Así se vería el ajuste con datos de numpy.

```python
model.fit(numpy_array_of_samples, numpy_array_of_labels,
          batch_size=32, epochs=10)
```

Además de los datos, se tiene especificar dos parámetros clave: el tamaño de los lotes, ```batch_size```, y el número de épocas, ```epochs``` (iteraciones en los datos). Aqui, los datos van a ser partidos en lotes de 32 muestras y el modelo iterará 10 veces sobre los datos durante el entrenamiento.

Aquí se muestra cómo se ajusta el modelo con el ```Dataset``` que se creo al inicio.

In [ ]:
history = model.fit(dataset, epochs = 2)

Epoch 1/2
196/196 [==============================] - 35s 172ms/step - loss: 4.9612
Epoch 2/2
196/196 [==============================] - 38s 187ms/step - loss: 4.8952


Como lo datos ya van a ser suministrados en forma de lotes, no es necesario especificar el tamaño de lote aqui.

El método fit() regresa un objeto "history" que almacena qué pasó durante el curso del entrenamiento. El diccionario```history.history``` contiene una serie de tiempo por épocas de los valores de las métricas.

In [ ]:
print(history.history)  

{'loss': [4.961152076721191, 4.8952107429504395]}


## Seguimiento de las métricas de rendimiento

Conforme se entrena el modelo, se puede hacer un seguimiento de las metricas tales como precisión de la clasificación, recall, AUC, etc. Además, es pueden monitorear no sólo en el dataset de entrenamiento, sino también en el de validación.

## Monitoreo de las métricas

Se puede pasar una lista de objetos de métricas a ```compile()``` así:

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)
history = model.fit(dataset, epochs=1)


196/196 [==============================] - 38s 184ms/step - loss: 4.8229 - acc: 0.0461


In [ ]:
history.history

{'acc': [0.046140000224113464], 'loss': [4.822906970977783]}

## Pasando datos de validación a ```fit()```

Se pueden pasat los datos de validación a fir para monitorear las métricas en este dataset. La métricas de validación se reportan al final de cada época.

In [ ]:
val_dataset = keras.preprocessing.image_dataset_from_directory(
  '/content/tiny-imagenet-200/val', batch_size=512, image_size=(64, 64))

Found 10000 files belonging to 1 classes.


In [ ]:
history = model.fit(dataset, epochs=4, validation_data=val_dataset)

Epoch 1/4
196/196 [==============================] - 41s 203ms/step - loss: 4.7051 - acc: 0.0593 - val_loss: 6.9610 - val_acc: 0.0089
Epoch 2/4
196/196 [==============================] - 40s 200ms/step - loss: 4.6479 - acc: 0.0645 - val_loss: 7.0790 - val_acc: 0.0050
Epoch 3/4
196/196 [==============================] - 41s 204ms/step - loss: 4.5958 - acc: 0.0713 - val_loss: 7.4957 - val_acc: 0.0078
Epoch 4/4
196/196 [==============================] - 40s 200ms/step - loss: 4.5563 - acc: 0.0766 - val_loss: 7.2014 - val_acc: 0.0118


# Tarea opcional

Leer [Intro to Keras for Engineers](https://keras.io/getting_started/intro_to_keras_for_engineers/), tiene más temas interesantes que no se consideraron en este notebook.